In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import os
from model import *
from data import *
import numpy as np
import torch
from data import CLASSES

path = 'adv_classes'
test_file_new='new_testing_list.txt'
#val_file='validation_list.txt'
train_file='train_list.txt'

['jam_test', 'negatives', 'positives']


In [3]:
import torch
torch.cuda.is_available()

False

In [4]:
batch_size = 10
epochs = 100

model_tag = 'model_pre_{}_{}'.format(epochs, batch_size)
model_save_path = os.path.join('adv_models', model_tag)
if not os.path.exists(model_save_path):
    os.mkdir(model_save_path)

train_dataset = SolarDataset(train_file)
test_dataset = SolarDataset(test_file_new)
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True,
                              collate_fn=collate_fn, num_workers=4)
test_dataloader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True,
                             collate_fn=collate_fn, num_workers=4)

In [4]:
model = KWS()

In [5]:
model.fc[3]=nn.Linear(32,2)
print(model)

KWS(
  (cnn_extractor): Sequential(
    (0): Conv2d(1, 10, kernel_size=(5, 1), stride=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(10, 1, kernel_size=(5, 1), stride=(1, 1))
    (4): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (rnn): LSTM(1, 128, num_layers=2, batch_first=True, bidirectional=True)
  (q_emb): Linear(in_features=256, out_features=1024, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=2, bias=True)
  )
)


In [6]:
model.load_state_dict(torch.load("./adv_models/model_pre_100_10/epoch_95.pth",map_location=torch.device('cpu')))


<All keys matched successfully>

In [7]:
model.fc[3]=nn.Linear(32,2)
print(model)

KWS(
  (cnn_extractor): Sequential(
    (0): Conv2d(1, 10, kernel_size=(5, 1), stride=(1, 1))
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(10, 1, kernel_size=(5, 1), stride=(1, 1))
    (4): BatchNorm2d(1, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): AvgPool2d(kernel_size=2, stride=2, padding=0)
  )
  (rnn): LSTM(1, 128, num_layers=2, batch_first=True, bidirectional=True)
  (q_emb): Linear(in_features=256, out_features=1024, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=2, bias=True)
  )
)


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
from tqdm import tqdm
import logging
logging.basicConfig(filename='log.log',level=logging.INFO)

print_step = 10

for epoch in range(epochs):
    pbar = tqdm(train_dataloader)
    total_loss = 0
    total_sample = 0
    correct = 0
    model.train()
    cur_step = 0
    logging.info("[training]training start")
    for batch in pbar:
        optimizer.zero_grad()
        x, y = batch

        x = x
        y = y
        
        logits = model(x)
        
        loss = criterion(logits,y)

        total_loss += loss.item()
        total_sample += y.size(0)
        
        pred = logits.data.max(1, keepdim=True)[1]
        correct += pred.eq(y.data.view_as(pred)).sum()

        loss.backward()
        optimizer.step()
        cur_step += 1
        pbar.set_description("loss: {}\tacc:{}\t".format(total_loss/cur_step, (correct/total_sample)*100))
        
        if cur_step%print_step==0:
            logging.info("[training]epoch {}\tloss: {}\tacc:{}\t".format(epoch,total_loss/cur_step, (correct/total_sample)*100))
    
    
    if epoch%5==0:
        torch.save(model.state_dict(), os.path.join(model_save_path, 'epoch_{}.pth'.format(epoch)))


loss: 0.15610298880980866	acc:97.06745147705078	: 100%|██████████| 35/35 [06:37<00:00, 11.34s/it]
loss: 0.007076886414662502	acc:99.70674133300781	: 100%|██████████| 35/35 [07:50<00:00, 13.43s/it]
loss: 0.002395669776316416	acc:100.0	: 100%|██████████| 35/35 [07:37<00:00, 13.09s/it] 
loss: 0.021680363020797292	acc:99.70674133300781	: 100%|██████████| 35/35 [07:31<00:00, 12.91s/it]
loss: 0.024534875282526108	acc:99.70674133300781	: 100%|██████████| 35/35 [07:20<00:00, 12.59s/it]
loss: 0.0037796719791263708	acc:100.0	: 100%|██████████| 35/35 [07:29<00:00, 12.83s/it]
loss: 0.025661328162771366	acc:99.41349029541016	: 100%|██████████| 35/35 [07:28<00:00, 12.82s/it]
loss: 0.00044662547720041143	acc:100.0	: 100%|██████████| 35/35 [06:44<00:00, 11.55s/it]
loss: 0.022032341416964602	acc:99.58333587646484	:  69%|██████▊   | 24/35 [06:26<00:35,  3.23s/it]

In [7]:
test_file_new='new_testing_list.txt'


In [8]:

with open(os.path.join(path,test_file_new)) as f:
    test_lines = f.readlines()
    
 

In [9]:
test_lines_new=[]
for line in test_lines:
        test_lines_new.append(path+'/'+line)
        


In [14]:

for i in range(0,2376):
    print(test_lines_new[i][65:80])

 2 pos 12.wav

 2 pos 14.wav

 2 pos 16.wav

 2 pos 18.wav

 2 pos 20.wav

 2 pos 22.wav

 2 pos 24.wav

 2 pos 26.wav

 2 pos 28.wav

 2 pos 30.wav

 2 pos 32.wav

 2 pos 34.wav

 2 pos 36.wav

 2 pos 38.wav

 2 pos 40.wav

 2 pos 42.wav

 2 pos 44.wav

 2 pos 46.wav

 2 pos 48.wav

 2 pos 50.wav

 2 pos 52.wav

 2 pos 54.wav

 2 pos 56.wav

 2 pos 58.wav

 2 pos 60.wav

 2 pos 62.wav

 2 pos 64.wav

 2 pos 66.wav

 2 pos 68.wav

 2 pos 70.wav

 2 pos 72.wav

 2 pos 74.wav

 2 pos 76.wav

 2 pos 78.wav

 2 pos 80.wav

 2 pos 82.wav

 2 pos 84.wav

 2 pos 86.wav

 2 pos 88.wav

 2 pos 90.wav

 2 pos 92.wav

 2 pos 94.wav

 2 pos 96.wav

 2 pos 98.wav

 2 pos 100.wav

 2 pos 102.wav

 2 pos 104.wav

 2 pos 106.wav

 2 pos 108.wav

 2 pos 110.wav

 2 pos 112.wav

 2 pos 114.wav

 2 pos 116.wav

 2 pos 118.wav

 2 pos 120.wav

 2 pos 122.wav

 2 pos 124.wav

 2 pos 126.wav

 2 pos 128.wav

 2 pos 130.wav

 2 pos 132.wav

 2 pos 134.wav

 2 pos 136.wav

 2 pos 138.wav

 2 pos 140.wav

 2 p

In [15]:
batch_size = 1
test_dataset = SolarDataset(test_file_new)
test_dataloader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True,
                             collate_fn=collate_fn, num_workers=4)

In [16]:
test_file_new

'new_testing_list.txt'

In [17]:
with torch.no_grad():
        pbar = test_dataloader
        total_loss = 0
        total_sample = 0
        correct = 0
        cur_step = 0
        model.eval()
        for x,y in pbar:
            logits = model(x)
            pred_index = logits.data.max(1, keepdim=True)[1]
            pred_val=logits.data.max(1,keepdim=True)[0]
            print(pred_val)


tensor([[3.1261]])
tensor([[2.8063]])
tensor([[3.0709]])
tensor([[2.7917]])
tensor([[2.7256]])
tensor([[2.6528]])
tensor([[2.2292]])
tensor([[2.1987]])
tensor([[2.3850]])
tensor([[2.5349]])
tensor([[2.6905]])
tensor([[2.7275]])
tensor([[2.7637]])
tensor([[2.8567]])
tensor([[2.9872]])
tensor([[3.0369]])
tensor([[2.7961]])
tensor([[2.6237]])
tensor([[2.4116]])
tensor([[2.0292]])
tensor([[2.2412]])
tensor([[2.1373]])
tensor([[2.1931]])
tensor([[2.0709]])
tensor([[2.0391]])
tensor([[1.7878]])
tensor([[1.6839]])
tensor([[1.5352]])
tensor([[1.4128]])
tensor([[1.2056]])
tensor([[1.0320]])
tensor([[0.8176]])
tensor([[0.6812]])
tensor([[0.7139]])
tensor([[0.6909]])
tensor([[0.6240]])
tensor([[0.6026]])
tensor([[0.5519]])
tensor([[0.4646]])
tensor([[0.4554]])
tensor([[0.4681]])
tensor([[0.7444]])
tensor([[1.0368]])
tensor([[0.8144]])
tensor([[1.1535]])
tensor([[1.2616]])
tensor([[1.8216]])
tensor([[2.2782]])
tensor([[2.2086]])
tensor([[2.4020]])
tensor([[2.2782]])
tensor([[2.0725]])
tensor([[1.9

tensor([[0.9352]])
tensor([[0.9235]])
tensor([[0.8929]])
tensor([[0.9720]])
tensor([[1.0268]])
tensor([[1.0839]])
tensor([[1.2729]])
tensor([[1.1806]])
tensor([[1.1128]])
tensor([[1.0120]])
tensor([[1.0647]])
tensor([[1.0140]])
tensor([[1.0020]])
tensor([[1.0740]])
tensor([[0.9396]])
tensor([[1.0541]])
tensor([[1.0647]])
tensor([[1.0862]])
tensor([[1.2597]])
tensor([[1.3348]])
tensor([[1.3936]])
tensor([[1.6017]])
tensor([[1.4823]])
tensor([[1.6307]])
tensor([[1.6067]])
tensor([[1.6349]])
tensor([[1.6409]])
tensor([[1.6742]])
tensor([[1.7185]])
tensor([[1.7002]])
tensor([[1.7161]])
tensor([[1.6992]])
tensor([[1.7207]])
tensor([[1.6993]])
tensor([[1.7114]])
tensor([[1.7121]])
tensor([[1.6979]])
tensor([[1.7201]])
tensor([[1.7494]])
tensor([[1.7051]])
tensor([[1.6878]])
tensor([[1.6927]])
tensor([[1.6866]])
tensor([[1.6253]])
tensor([[1.5952]])
tensor([[1.5684]])
tensor([[1.5420]])
tensor([[1.5222]])
tensor([[1.4696]])
tensor([[1.4492]])
tensor([[1.3541]])
tensor([[1.2641]])
tensor([[1.1

tensor([[0.6631]])
tensor([[0.7291]])
tensor([[0.7972]])
tensor([[0.8374]])
tensor([[0.8772]])
tensor([[0.9167]])
tensor([[0.9450]])
tensor([[0.9814]])
tensor([[1.0080]])
tensor([[1.0273]])
tensor([[1.0471]])
tensor([[1.0520]])
tensor([[1.0338]])
tensor([[0.9980]])
tensor([[0.9623]])
tensor([[0.9003]])
tensor([[0.8849]])
tensor([[3.4436]])
tensor([[3.3680]])
tensor([[3.6459]])
tensor([[3.6043]])
tensor([[3.5172]])
tensor([[3.3915]])
tensor([[3.3499]])
tensor([[3.3002]])
tensor([[3.2076]])
tensor([[3.1633]])
tensor([[3.1283]])
tensor([[2.9746]])
tensor([[2.7848]])
tensor([[2.5696]])
tensor([[2.7787]])
tensor([[3.1467]])
tensor([[3.4165]])
tensor([[3.6703]])
tensor([[4.0502]])
tensor([[4.1913]])
tensor([[4.2086]])
tensor([[4.1980]])
tensor([[4.0795]])
tensor([[4.0785]])
tensor([[3.9161]])
tensor([[3.8840]])
tensor([[3.6704]])
tensor([[3.7485]])
tensor([[3.5009]])
tensor([[3.4325]])
tensor([[2.9486]])
tensor([[1.8190]])
tensor([[1.8668]])
tensor([[1.6180]])
tensor([[1.6146]])
tensor([[1.6

tensor([[1.7607]])
tensor([[1.7476]])
tensor([[1.5865]])
tensor([[1.6313]])
tensor([[1.3176]])
tensor([[1.3145]])
tensor([[0.8064]])
tensor([[0.7932]])
tensor([[1.1513]])
tensor([[0.9835]])
tensor([[1.2907]])
tensor([[1.3661]])
tensor([[1.7884]])
tensor([[1.7125]])
tensor([[1.8535]])
tensor([[2.0547]])
tensor([[2.2365]])
tensor([[2.3415]])
tensor([[2.4178]])
tensor([[2.4349]])
tensor([[2.4139]])
tensor([[2.3637]])
tensor([[2.3509]])
tensor([[2.3834]])
tensor([[2.3318]])
tensor([[2.3409]])
tensor([[2.3160]])
tensor([[2.1756]])
tensor([[2.1395]])
tensor([[2.3948]])
tensor([[2.3735]])
tensor([[2.5647]])
tensor([[2.6249]])
tensor([[2.6203]])
tensor([[2.5811]])
tensor([[2.6220]])
tensor([[2.6227]])
tensor([[2.5614]])
tensor([[2.5249]])
tensor([[2.6117]])
tensor([[2.4940]])
tensor([[2.5222]])
tensor([[2.5186]])
tensor([[2.4540]])
tensor([[2.4918]])
tensor([[2.5133]])
tensor([[2.5865]])
tensor([[2.5635]])
tensor([[2.6164]])
tensor([[2.6534]])
tensor([[2.6168]])
tensor([[2.5931]])
tensor([[2.5

tensor([[1.7066]])
tensor([[1.7529]])
tensor([[1.5786]])
tensor([[1.5670]])
tensor([[1.5974]])
tensor([[1.5728]])
tensor([[1.4557]])
tensor([[1.4120]])
tensor([[1.3743]])
tensor([[1.2834]])
tensor([[1.1373]])
tensor([[1.0387]])
tensor([[0.9029]])
tensor([[0.7755]])
tensor([[0.6769]])
tensor([[0.6561]])
tensor([[0.6522]])
tensor([[0.6652]])
tensor([[0.6632]])
tensor([[0.7151]])
tensor([[0.7189]])
tensor([[0.5305]])
tensor([[0.4800]])
tensor([[0.4683]])
tensor([[0.4664]])
tensor([[0.4900]])
tensor([[0.4817]])
tensor([[0.5014]])
tensor([[0.5659]])
tensor([[0.6352]])
tensor([[0.7091]])
tensor([[0.7676]])
tensor([[0.8311]])
tensor([[0.8807]])
tensor([[0.9169]])
tensor([[0.9646]])
tensor([[1.0091]])
tensor([[1.0452]])
tensor([[1.0745]])
tensor([[1.0840]])
tensor([[1.0685]])
tensor([[1.0177]])
tensor([[0.9622]])
tensor([[0.9003]])
tensor([[0.8849]])
tensor([[3.3976]])
tensor([[3.4204]])
tensor([[3.3396]])
tensor([[3.2324]])
tensor([[3.0672]])
tensor([[3.2089]])
tensor([[2.9375]])
tensor([[3.1

tensor([[1.9291]])
tensor([[1.8814]])
tensor([[1.7994]])
tensor([[4.8671]])
tensor([[2.8996]])
tensor([[4.8833]])
tensor([[4.2076]])
tensor([[3.4696]])
tensor([[5.4786]])
tensor([[2.4136]])
tensor([[4.7583]])
tensor([[4.5085]])
tensor([[4.3238]])
tensor([[4.5275]])
tensor([[2.2588]])
tensor([[2.5695]])
tensor([[2.5810]])
tensor([[1.8286]])
tensor([[1.8560]])
tensor([[2.0858]])
tensor([[4.1342]])
tensor([[1.9324]])
tensor([[4.4494]])
tensor([[2.1962]])
tensor([[3.6502]])
tensor([[3.2567]])
tensor([[1.5192]])
tensor([[1.6660]])
tensor([[1.5946]])
tensor([[1.6271]])
tensor([[1.6347]])
tensor([[1.5784]])
tensor([[1.5450]])
tensor([[1.6457]])
tensor([[1.6464]])
tensor([[1.8041]])
tensor([[1.7880]])
tensor([[1.7905]])
tensor([[1.7085]])
tensor([[1.7880]])
tensor([[1.8242]])
tensor([[1.7646]])
tensor([[1.7291]])
tensor([[1.8352]])
tensor([[2.0789]])
tensor([[2.4869]])
tensor([[4.5739]])
tensor([[2.6969]])
tensor([[2.1131]])
tensor([[2.1539]])
tensor([[2.0419]])
tensor([[2.0658]])
tensor([[1.8

In [ ]:
import torch
import librosa
import numpy as np

In [ ]:
def load_audio(file_path):
    audio, sr = librosa.load(file_path, sr=16000)
    audio = audio[None, :]
    audio = torch.from_numpy(audio).float()
    return audio

In [ ]:
def predict(model, audio):
    model.eval()
    audio = audio
    logits = model(audio)
    data=transform()
    pred = logits.data.max(1, keepdim=True)[1]
    return int(pred.cpu().numpy()[0][0])


In [ ]:
logging.info("[test]test start")
with torch.no_grad():
    pbar = tqdm(test_dataloader)
    total_loss = 0
    total_sample = 0
    correct = 0
    cur_step = 0
    model.eval()
    for batch in pbar:
        x, y = batch

        x = x
        y = y

        logits = model(x)

        loss = criterion(logits,y)

        total_loss += loss.item()
        total_sample += y.size(0)

        pred = logits.data.max(1, keepdim=True)[1]
        cur_step += 1
        correct += pred.eq(y.data.view_as(pred)).sum()
print("[test]epoch {}\tloss: {}\tacc:{}\t".format(epoch,total_loss/cur_step, (correct/total_sample)*100))
logging.info("[test]epoch {}\tloss: {}\tacc:{}\t".format(epoch,total_loss/cur_step, (correct/total_sample)*100))

In [ ]:
model=torch.load("./models/model_pre_100_16/epoch_99.pth",map_location=torch.device('cpu'))